In [1]:
# MIT image : Analysis
# Made by : HERO Kwon
# Date : 190113

from numba import jit
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
#import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
import time

# data path
path_csi = 'D:\\Data\\Wi-Fi_processed\\'
path_csi_np = 'D:\\Data\\Wi-Fi_processed_npy\\'
path_meta = 'D:\\Data\\Wi-Fi_meta\\'
path_sc = 'D:\\Data\\Wi-Fi_info\\'
path_mit_image = 'D:\\Data\\Wi-Fi_mit_image\\'

In [2]:
# data info
df_info = pd.read_csv(path_meta+'data_subc_sig_v1.csv')
df_info = df_info[df_info.id_location==1]

person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))
csi_time = 50 #15000 #int(np.max(df_info['len']))
# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# freq BW list
bw_list = pd.read_csv(path_meta+'wifi_f_bw_list.csv')

# 3D scan param
m,n = 2,3
c =  299792458 # speed of light 
#r = (160 + 160 + 164) * 0.01 # meter
r = 1.64 #meter
d = 45 * 0.01 # meter
ch = 8#3
max_subc = 30

# Load data
saved_images = [f.replace(".npy","") for f in os.listdir(path_mit_image)]
saved_images

df11 = df_info[(df_info.id_direction==1) & (df_info.id_location==1)]
df_target = df11[df11['id'].isin(saved_images)]
df_lab = df_target[['id','id_person','id_location','id_direction','id_exp']]
df_lab = df_lab.drop_duplicates()
df_lab


,id,id_person,id_location,id_direction,id_exp
256,S018_1_1_1,18,1,1,1
262,S018_1_1_10,18,1,1,10
268,S018_1_1_11,18,1,1,11
274,S018_1_1_12,18,1,1,12
280,S018_1_1_2,18,1,1,2
286,S018_1_1_3,18,1,1,3
292,S018_1_1_5,18,1,1,5
298,S018_1_1_6,18,1,1,6
304,S018_1_1_7,18,1,1,7
310,S018_1_1_8,18,1,1,8


In [3]:
from scipy.misc import imresize

In [4]:
data_mit = []
filename_mit = []
for file in df_lab.id.values:
    data_load = np.load(path_mit_image + file + '.npy')
    data_mit.append(data_load)
    filename_mit.append(file)

arr_mit = np.array(data_mit)
sum_mit = np.sum(arr_mit,axis=1).reshape([-1,20*20])
norm_mit = np.array([sum_mit[a,:] / np.max(sum_mit[a]) for a in range(sum_mit.shape[0])])

x_mit= imresize(norm_mit,[858,100])

label_mit = df_lab.id_person.values

C:\Users\herok\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


In [5]:
x_mit.shape

(858, 100)

In [6]:
# Basis : RM2
@jit
def RMmodel(order,X):
    m,l = X.shape
    M1 = []
    M2 = []
    M3 = []
    MM1 = []
    MM3 = []
    Msum = np.sum(X,axis=1)
    for i in range(order):
        for k in range(l):
            M1.append(X[:,k]**(i+1))
            if (i>0):
                M3.append(X[:,k]*Msum**(i)) 
        M2.append(Msum**(i+1))
        MM1.append(M1)
        if (i>0):
            MM3.append(M3)
    MM1 = np.array(MM1).T
    MM1 = MM1.reshape((m,-1,1)).squeeze(axis=2)
    M2 = np.array(M2).T
    if (len(MM3)):
        MM3 = np.array(MM3).T
        MM3 = MM3.reshape((m,-1,1)).squeeze(axis=2)
        P = np.concatenate((np.ones((m,1)),MM1,M2,MM3),axis=1)
    else : P = np.concatenate((np.ones((m,1)),MM1,M2),axis=1)
    return(P)

#Models : LSE,RM,TER
@jit
def Model3(rank,r,n,X,Y,mod_w):
    # LSE
    alpha_lse = []
    for k in list(set(Y)):
        P_lse = X
        I_lse = np.eye(P_lse.shape[1])
        b_lse = 10**(-4)
        y_lse = (Y==k).astype('int')
        ak_lse = np.dot(np.dot(np.linalg.pinv(b_lse*I_lse + P_lse.T.dot(P_lse)),P_lse.T),y_lse)
        alpha_lse.append(ak_lse)    
    # RM
    alpha_rm = []
    for k in list(set(Y)):
        P_rm = RMmodel(rank,X)
        I_rm = np.eye(P_rm.shape[1])
        b_rm = 10**(-4)
        y_rm = (Y==k).astype('int')
        ak_rm = np.dot(np.dot(np.linalg.pinv(b_rm*I_rm + P_rm.T.dot(P_rm)),P_rm.T),y_rm)
        alpha_rm.append(ak_rm)  
    # TER
    alpha_ter = []
    for k in list(set(Y)):
        P_n = RMmodel(rank,X[Y!=k])
        P_p = RMmodel(rank,X[Y==k])
        mk_n = X[Y!=k].shape[0]
        mk_p = X[Y==k].shape[0]
        yk_n = (r-n) * np.ones(shape=Y[Y!=k].shape)
        yk_p = (r+n) * np.ones(shape=Y[Y==k].shape)
        I = np.eye(P_n.shape[1])
        b = 10**(-4)
        first_eq = np.linalg.pinv(b*I + (1/mk_n)*(P_n.T).dot(P_n) + (1/mk_p)*(P_p.T).dot(P_p))
        second_eq = (1/mk_n)*(P_n.T).dot(yk_n) + (1/mk_p)*(P_p.T).dot(yk_p)
        ak = np.dot(first_eq,second_eq)
        alpha_ter.append(ak)
        
    # new TER
    #simplified version of TER
    alpha_nter = []
    for k in list(set(Y)):
        P_nter = RMmodel(rank,X)

        mk_n_nter = X[Y!=k].shape[0]
        mk_p_nter = X[Y==k].shape[0]
        w_n_nter = 1/mk_n_nter
        w_p_nter = 1/mk_p_nter

        w_n_nter = mod_w[0][0] * w_n_nter + mod_w[0][1]
        w_p_nter = mod_w[1][0] * w_n_nter + mod_w[1][1]

        ones_mkn_nter = 1*(Y!=k) *w_n_nter
        ones_mkp_nter = 1*(Y==k) *w_p_nter

        W_nter = np.zeros((len(Y), len(Y)), float)
        np.fill_diagonal(W_nter,ones_mkn_nter+ones_mkp_nter)
        yk_nter = ((r-n)*ones_mkn_nter+(r+n)*ones_mkp_nter).T
        ak_nter = np.linalg.pinv((P_nter.T).dot(W_nter).dot(P_nter)).dot(P_nter.T).dot(W_nter).dot(yk_nter)

        alpha_nter.append(ak_nter)
    return(np.array(alpha_lse).T, np.array(alpha_rm).T,np.array(alpha_ter).T,np.array(alpha_nter).T)

In [7]:
# change label
uniq_label = np.unique(label_mit)
label_table = pd.Series(np.arange(len(uniq_label)),index=uniq_label)
label_num = np.array([label_table[num] for num in label_mit])

# K-Fold training data
skf = StratifiedKFold(n_splits=5)
split = 0
data_v = x_mit
data_l = label_num
for train_index,test_index in skf.split(data_v,data_l):
    data_train,data_test = arr_mit[train_index],arr_mit[test_index]
    label_train,label_test = label_mit[train_index],label_mit[test_index]

C:\Users\herok\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [8]:
M_dict = {}
M_dict['0'] = (0,1),(0,1)
M_dict['1'] = (1,0),(1,0)
M_dict['2'] = (1,0),(0,1)
M_dict['3'] = (0,1),(1,0)
M_dict['4'] = (1,0),(0.5,0)
M_dict['5'] = (0.5,0),(1,0)
M_dict['6'] = (1,0),(0.7,0)
M_dict['7'] = (0.7,0),(1,0)
M_dict['8'] = (1,0),(0.3,0)
M_dict['9'] = (0.3,0),(1,0)

In [9]:
df_acc = pd.DataFrame()

# iterate Split data
for train_index,test_index in skf.split(data_v,data_l):
    data_train,data_test = data_v[train_index],data_v[test_index]
    label_train,label_test = data_l[train_index],data_l[test_index]
    # iterate Rank 1~5    
    for j in range(10):
        rank = 3
        #Traning Result of models
        alpha_lse, alpha_rm, alpha_ter,alpha_nter = Model3(rank,0.5,0.5,data_train,label_train,M_dict[str(j)])
        #Test: LSE
        time_now = time.time()
        Pt_lse = data_test
        yt_lse = Pt_lse.dot(alpha_lse)
        yt1_lse = np.argmax(yt_lse,axis=1)
        pred_true = np.equal(label_test,yt1_lse)
        acc = np.count_nonzero(pred_true) / len(pred_true)
        time_elapsed = time.time() - time_now
        res_ser = pd.Series(['Data','LSE',split,0,acc,0],index=['Data','Model','Split','Rank','Acc','M'])
        df_acc = df_acc.append(res_ser,ignore_index=True)
        #Test: RM
        time_now = time.time()
        Pt_rm = RMmodel(rank,data_test)
        yt_rm = Pt_rm.dot(alpha_rm)
        yt1_rm = np.argmax(yt_rm,axis=1)
        pred_true = np.equal(label_test,yt1_rm)
        acc = np.count_nonzero(pred_true) / len(pred_true)
        time_elapsed = time.time() - time_now
        res_ser = pd.Series(['Data','RM',split,rank,acc,0],index=['Data','Model','Split','Rank','Acc','M'])
        df_acc = df_acc.append(res_ser,ignore_index=True)        
        #Test: TER
        time_now = time.time()
        Pt_ter = RMmodel(rank,data_test)
        yt_ter = Pt_ter.dot(alpha_ter)
        yt1_ter = np.argmax(yt_ter,axis=1)
        pred_true = np.equal(label_test,yt1_ter)
        acc = np.count_nonzero(pred_true) / len(pred_true)
        time_elapsed = time.time() - time_now
        res_ser = pd.Series(['Data','TER',split,rank,acc,0],index=['Data','Model','Split','Rank','Acc','M'])
        df_acc = df_acc.append(res_ser,ignore_index=True)
        #Test: nTER
        time_now = time.time()
        Pt_ter_n = RMmodel(rank,data_test)
        yt_ter_n = Pt_ter_n.dot(alpha_nter)
        yt1_ter_n = np.argmax(yt_ter_n,axis=1)
        pred_true = np.equal(label_test,yt1_ter_n)
        acc = np.count_nonzero(pred_true) / len(pred_true)
        time_elapsed = time.time() - time_now
        res_ser = pd.Series(['Data','nTER',split,rank,acc,j],index=['Data','Model','Split','Rank','Acc','M'])
        df_acc = df_acc.append(res_ser,ignore_index=True)
    print('Data:' + '/ Split:'+ str(split))
    split += 1

C:\Users\herok\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Data:/ Split:0
Data:/ Split:1
Data:/ Split:2
Data:/ Split:3
Data:/ Split:4


In [14]:
df_acc
#df_acc.to_csv('df_acc_nter.csv')

In [19]:
df_res = df_acc.groupby(['Model','M']).mean()
df_res.to_csv('df_res_nter.csv')

In [12]:
#df_res.to_csv('df_res.csv')

In [20]:
df_res

Acc  Rank  Split
Model M                         
LSE   0.0  0.008856   0.0    2.0
RM    0.0  0.081994   3.0    2.0
TER   0.0  0.069943   3.0    2.0
nTER  0.0  0.081994   3.0    2.0
      1.0  0.081994   3.0    2.0
      2.0  0.074411   3.0    2.0
      3.0  0.081994   3.0    2.0
      4.0  0.085355   3.0    2.0
      5.0  0.081994   3.0    2.0
      6.0  0.085223   3.0    2.0
      7.0  0.081994   3.0    2.0
      8.0  0.087233   3.0    2.0
      9.0  0.081994   3.0    2.0